In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from glob import glob
import re

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams["font.family"] = "DejaVu Sans"

# Carpeta que contiene los archivos Excel
carpeta = "."
archivos = glob(os.path.join(carpeta, "*.xlsx"))

# Función para extraer número de la pregunta para ordenamiento
def extraer_numero(pregunta):
    match = re.search(r'\d+', str(pregunta))
    return int(match.group()) if match else float('inf')

# Carpeta de salida para gráficos
output_folder = "graficos"
os.makedirs(output_folder, exist_ok=True)

# Procesar cada archivo
for archivo in archivos:
    try:
        df = pd.read_excel(archivo)
        nombre_archivo = os.path.basename(archivo)
        nombre_base = os.path.splitext(nombre_archivo)[0]
        titulo_limpio = re.sub(r"[_\-]+", " ", nombre_base).title()

        # Verificación básica
        if "Apellido(s)" not in df.columns or "Nombre" not in df.columns:
            print(f"[{nombre_archivo}] Archivo inválido. Saltando.")
            continue

        df["Nombre completo"] = df["Apellido(s)"] + " " + df["Nombre"]

        # Detectar columnas de preguntas
        preguntas_cols = [col for col in df.columns if col.startswith("P.") or col.startswith("P ")]

        # Obtener puntajes máximos por pregunta desde el encabezado
        max_puntaje_por_pregunta = {}
        for col in preguntas_cols:
            match = re.search(r'/\s*([\d.,]+)', col)
            if match:
                valor_raw = match.group(1).replace(",", ".")
                try:
                    max_puntaje_por_pregunta[col] = float(valor_raw)
                except ValueError:
                    max_puntaje_por_pregunta[col] = 1.0  # por defecto
            else:
                max_puntaje_por_pregunta[col] = 1.0  # por defecto

        # Despivotar (unpivot)
        df_unpivot = df.melt(
            id_vars=["Nombre completo"],
            value_vars=preguntas_cols,
            var_name="Pregunta",
            value_name="Respuesta"
        )

        # Clasificación de respuestas
        def clasificar(row):
            valor = str(row["Respuesta"]).strip()

            if valor in ["-", ""]:
                return "Sin contestar"

            try:
                valor = valor.replace(",", ".")
                puntaje = float(valor)
            except ValueError:
                return "Incorrecta"  # Tratamos valores no numéricos como incorrectos

            pregunta = row["Pregunta"]
            max_puntaje = max_puntaje_por_pregunta.get(pregunta, 1.0)

            if puntaje == max_puntaje:
                return "Correcta"
            elif 0 < puntaje < max_puntaje:
                return "Parcialmente correcta"
            elif puntaje == 0:
                return "Incorrecta"
            else:
                return "Incorrecta"  # Puntajes fuera de rango

        df_unpivot["Respuesta Tipo"] = df_unpivot.apply(clasificar, axis=1)

        # Agrupar para el gráfico
        categorias_fijas = ["Correcta", "Parcialmente correcta", "Incorrecta", "Sin contestar"]
        conteo = (
            df_unpivot.groupby(["Pregunta", "Respuesta Tipo"])
            .size()
            .unstack()
            .reindex(columns=categorias_fijas, fill_value=0)
        )
        conteo = conteo.loc[sorted(conteo.index, key=extraer_numero)]

        # Graficar
        colores = {
            "Correcta": "#4CAF50",
            "Parcialmente correcta": "#FFC107",
            "Incorrecta": "#F44336",
            "Sin contestar": "#9E9E9E"
        }

        ax = conteo.plot(
            kind="bar",
            stacked=True,
            figsize=(14, 6),
            title=f"Respuestas por Pregunta - {titulo_limpio}",
            color=colores
        )
        plt.xlabel("Pregunta")
        plt.ylabel("Cantidad de respuestas")
        plt.legend(
            title="Tipo de respuesta",
            bbox_to_anchor=(1.05, 1),
            loc='upper left',
            borderaxespad=0.
        )
        plt.tight_layout(rect=[0, 0, 0.85, 1])

        # Guardar gráfico
        output_path = os.path.join(output_folder, f"{nombre_base}.png")
        plt.savefig(output_path)
        plt.close()
        print(f"[{nombre_archivo}] Gráfico generado en: {output_path}")

    except Exception as e:
        print(f"[{archivo}] Error al procesar: {e}")


[1 - CIENCIAS SOCIALES SEGUNDO CICLO-calificaciones (1).xlsx] Gráfico generado en: graficos/1 - CIENCIAS SOCIALES SEGUNDO CICLO-calificaciones (1).png
[1 MATEMÁTICAS- SEGUNDO CICLO-calificaciones.xlsx] Gráfico generado en: graficos/1 MATEMÁTICAS- SEGUNDO CICLO-calificaciones.png
[1 LENGUAJE- SEGUNDO CICLO-calificaciones.xlsx] Gráfico generado en: graficos/1 LENGUAJE- SEGUNDO CICLO-calificaciones.png
[Evaluación 1 MATEMATICA- BASICA-calificaciones.xlsx] Gráfico generado en: graficos/Evaluación 1 MATEMATICA- BASICA-calificaciones.png
[1 MATEMÁTICAS- PRIMER CICLO-calificaciones.xlsx] Gráfico generado en: graficos/1 MATEMÁTICAS- PRIMER CICLO-calificaciones.png
[1 Ciencias Naturales-Primer Ciclo Media-calificaciones.xlsx] Gráfico generado en: graficos/1 Ciencias Naturales-Primer Ciclo Media-calificaciones.png
[EVALUACIÓN 1 CIENCIAS NATURALES-BASICA-calificaciones.xlsx] Gráfico generado en: graficos/EVALUACIÓN 1 CIENCIAS NATURALES-BASICA-calificaciones.png
[1 LENGUAJE- PRIMER CICLO-calificac